In [1]:
# Notebook for visualising breakthrough infections in a different way
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


# Load data

In [36]:
dfPop = pd.read_excel('DKfolketal2021clean.xlsx')
dfPop.head()


700466

In [3]:
ssidatapath = "ssi_vacc"
rootdir = os.getcwd() +"\\" + ssidatapath



for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])
    
print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-09-20
2021-09-20 00:00:00


In [16]:

filename = '/Vaccine_DB/Vaccinationer_region_aldgrp_koen.csv'
curPath = latestdir+ filename

df_vacc = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')
df_vacc[df_vacc['Aldersgruppe'] == '20-29']['Antal faerdigvacc.'].astype(int).sum()

567024

In [58]:
# We have to collect from previous dates

curAgeGroup = '20-29'
curPopSize = dfPop[20:29].Antal.sum()

agedf = pd.DataFrame()

allDates = []
allVacc = []

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])
    
    try:
        filename = '/Vaccine_DB/Vaccinationer_region_aldgrp_koen.csv'
        curPath = latestdir+ filename
        df_vacc = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')
        curVacc = df_vacc[df_vacc['Aldersgruppe'] == '20-29']['Antal faerdigvacc.'].astype(int).sum()
        
        allDates.append(latestDate)
        allVacc.append(curVacc)
    except:
        2+2
        
allDates = np.array(allDates)    
allVacc = np.array(allVacc)

agedf['Dates'] = allDates
agedf['Vaccinated'] = allVacc
agedf['Unvaccinated'] = curPopSize - allVacc
agedf.tail()

,Dates,Vaccinated,Unvaccinated
66,2021-09-16,563620,136846
67,2021-09-17,564552,135914
68,2021-09-18,565584,134882
69,2021-09-19,566396,134070
70,2021-09-20,567024,133442


In [55]:
# df_vacc
# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.fill_between(allDates,np.ones(allVacc.shape)*curPopSize,color='xkcd:light red')
ax1.fill_between(allDates,allVacc,color='xkcd:blue')
ax2.fill_between(allDates,100*np.ones(allVacc.shape),color='xkcd:light red')
ax2.fill_between(allDates,100*allVacc/curPopSize,color='xkcd:blue')

ax1.set_ylim(bottom=0,top=curPopSize)
ax2.set_ylim(bottom=0,top=100)

ax1.set_xlim([allDates.min(),allDates.max()])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
agedf.tail(20)

,Dates,Vaccinated,Unvaccinated
51,2021-09-01,541264,159202
52,2021-09-02,543458,157008
53,2021-09-03,545472,154994
54,2021-09-04,547267,153199
55,2021-09-05,548776,151690
56,2021-09-06,549829,150637
57,2021-09-07,551729,148737
58,2021-09-08,553583,146883
59,2021-09-09,555012,145454
60,2021-09-10,556257,144209


In [65]:
# Define datapath and load
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"\\" + ssidatapath


# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
tuePath = 'overvaagningsdata-covid19-14092021-jrhb'
tuePath = 'SSI_data_2021-09-14'

dirPath = rootdir + '\\' + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')
df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';')
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';')
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';')

In [66]:
# Flip dataframe to be like df1
casedf = pd.DataFrame()
casedf_once = pd.DataFrame()
casedf_both = pd.DataFrame()
casedf_full = pd.DataFrame()
casedf_none = pd.DataFrame()

weekNames = df1.Ugenummer
casedf['Ugenummer'] = weekNames
casedf_none['Ugenummer'] = weekNames
casedf_once['Ugenummer'] = weekNames
casedf_both['Ugenummer'] = weekNames
casedf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_C)):
    curRow = df2_C.loc[k]
    curAge = df2_C.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    casedf[curAge+'_Ingen vaccination'] = curNone
    casedf[curAge+'_Første vaccination'] = curOnce
    casedf[curAge+'_Anden vaccination'] = curBoth 
    casedf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    casedf_none[curAge] = curNone
    casedf_once[curAge] = curOnce
    casedf_both[curAge] = curBoth 
    casedf_full[curAge] = curFull
    

# df2_C
# curRow
casedf
# cur

12-15
16-19
20-29
30-39
40-49
50-59
60-69
70-79
80+
12+


,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 36,271,25,11,4,115,7,28,23,313,...,118,118,3,0,76,76,1251,174,1100,1070
1,Uge 35,441,53,14,10,158,41,44,39,391,...,128,128,12,2,75,74,1746,270,1476,1420
2,Uge 34,680,99,22,9,248,53,70,56,579,...,163,163,8,1,74,74,2570,469,1699,1583
3,Uge 33,703,117,20,0,323,69,87,70,792,...,117,117,3,0,71,71,2973,664,1661,1437
4,Uge 32,516,103,4,0,389,117,94,59,1045,...,103,103,7,2,55,54,3102,1075,1569,1242
5,Uge 31,439,63,0,0,360,128,84,44,1008,...,81,81,1,0,41,41,2877,1352,1324,1036
6,Uge 30,344,33,0,0,265,173,85,35,1035,...,59,59,1,0,31,31,2507,1945,1199,856
7,Uge 29,300,13,0,0,235,207,61,23,897,...,47,47,1,0,32,32,2101,1946,805,586
8,Uge 28,234,0,0,0,239,329,93,11,1336,...,40,40,2,0,20,20,2775,2805,856,597
9,Uge 27,142,0,0,0,183,264,19,4,986,...,30,30,1,0,9,9,2131,1791,444,350


In [69]:
casedf_none
# Uge 35 startede 30/08-2021

,Ugenummer,12-15,16-19,20-29,30-39,40-49,50-59,60-69,70-79,80+,12+
0,Uge 36,271,115,313,290,139,82,30,8,3,1251
1,Uge 35,441,158,391,386,218,99,33,8,12,1746
2,Uge 34,680,248,579,522,330,143,46,14,8,2570
3,Uge 33,703,323,792,652,297,152,38,13,3,2973
4,Uge 32,516,389,1045,648,304,149,40,4,7,3102
5,Uge 31,439,360,1008,598,298,143,25,5,1,2877
6,Uge 30,344,265,1035,515,221,94,30,2,1,2507
7,Uge 29,300,235,897,460,130,61,13,4,1,2101
8,Uge 28,234,239,1336,737,129,77,16,5,2,2775
9,Uge 27,142,183,986,635,119,43,19,3,1,2131


In [70]:
# Flip dataframe to be like df1
hospdf = pd.DataFrame()
hospdf_once = pd.DataFrame()
hospdf_both = pd.DataFrame()
hospdf_full = pd.DataFrame()
hospdf_none = pd.DataFrame()

weekNames = df1.Ugenummer
hospdf['Ugenummer'] = weekNames
hospdf_none['Ugenummer'] = weekNames
hospdf_once['Ugenummer'] = weekNames
hospdf_both['Ugenummer'] = weekNames
hospdf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_H)):
    curRow = df2_H.loc[k]
    curAge = df2_H.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    hospdf[curAge+'_Ingen vaccination'] = curNone
    hospdf[curAge+'_Første vaccination'] = curOnce
    hospdf[curAge+'_Anden vaccination'] = curBoth 
    hospdf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    hospdf_none[curAge] = curNone
    hospdf_once[curAge] = curOnce
    hospdf_both[curAge] = curBoth 
    hospdf_full[curAge] = curFull
    

# df2_H
# curRow
hospdf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 36,2,0,0,0,0,0,0,0,18,...,13,13,3,0,20,20,77,8,58,57
1,Uge 35,4,0,0,0,3,0,0,0,17,...,12,12,9,2,19,18,121,13,46,44
2,Uge 34,3,1,0,0,3,0,1,1,19,...,22,22,5,0,19,19,120,11,59,59
3,Uge 33,0,0,0,0,5,0,0,0,32,...,16,16,3,1,12,12,136,13,58,57
4,Uge 32,3,0,0,0,5,1,1,1,23,...,8,8,4,0,11,11,107,29,37,34
5,Uge 31,1,0,0,0,3,1,0,0,17,...,6,6,1,0,15,15,65,14,36,34
6,Uge 30,1,0,0,0,0,1,0,0,20,...,6,6,0,0,5,5,63,20,23,22
7,Uge 29,1,0,0,0,0,0,0,0,20,...,2,2,1,0,8,8,64,11,18,17
8,Uge 28,0,0,0,0,0,0,0,0,16,...,7,7,0,0,4,4,60,15,17,17
9,Uge 27,1,0,0,0,1,0,0,0,14,...,4,4,1,0,1,1,38,11,8,6


In [88]:

# import datetime
# datetime.date(2021, 8, 30).isocalendar()[1]

iniIndex = 0
endIndex = 1
numCaseUnvacc = hospdf_none.loc[iniIndex:endIndex,'20-29'].sum()
numCaseVacc = hospdf_full.loc[iniIndex:endIndex,'20-29'].sum()

iniDate = np.datetime64('2021-08-30')
endDate = iniDate + np.timedelta64(13,'D')

period_agedf = agedf[(agedf.Dates >= iniDate) & (agedf.Dates <= endDate)]

meanUnvacc = period_agedf['Unvaccinated'].mean()
meanVacc = period_agedf['Vaccinated'].mean()

print(f'Mellem {iniDate} og {endDate}, blev der:')
print(f'Indlagt {numCaseVacc} vaccinerede personer mellem 20 og 29, ud af {meanVacc:.0f} vaccinerede')
print(f'Indlagt {numCaseUnvacc} uvaccinerede personer mellem 20 og 29, ud af {meanUnvacc:.0f} uvaccinerede')
# print(f'- Ud af {meanVacc:.0f} vacccinerede personer, indlagt {numCaseVacc} personer')
# print(f'- Ud af {meanUnvacc:.0f} uvacccinerede personer, indlagt {numCaseUnvacc} personer')



Mellem 2021-08-30 og 2021-09-12, blev der:
Indlagt 3 vaccinerede personer mellem 20 og 29, ud af 548466 vaccinerede
Indlagt 35 uvaccinerede personer mellem 20 og 29, ud af 152000 uvaccinerede
